In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [2]:
#getting a data loader
from langchain_community.document_loaders import WebBaseLoader

1. Get Data Loader

In [3]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Roman_Empire")
data = loader.load()
data

[Document(page_content='\n\n\n\nRoman Empire - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1History\n\n\n\nToggle History subsection\n\n\n\n\n\n1.1Transition from Republic to Empire\n\n\n\n\n\n\n\n1.2Pax Romana\n\n\n\n\n\n\n\n1.3Transition from Classical 

2. Convert Data to Vector Database

In [4]:
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings

Chunking the documents

In [5]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [6]:
documents

[Document(page_content='Roman Empire - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1History\n\n\n\nToggle History subsection\n\n\n\n\n\n1.1Transition from Republic to Empire\n\n\n\n\n\n\n\n1.2Pax Romana\n\n\n\n\n\n\n\n1.3Transition from Classical to Late 

Creating the vector embeddings

In [7]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

3. Make a RAG pipeline

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [10]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

4. Building a Retriever

In [11]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [12]:
question = "what was the government like in roman empire?"
result = qa_chain({"query": question })
result

/Users/abhishekshah/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'what was the government like in roman empire?',
 'result': "The Roman Empire's government consisted of three major elements: the central government, the military, and the provincial government. The central government included elected magistrates and appointed officials who managed different aspects of state affairs, while the military maintained control and order. Provincial governance involved local elites and officials to ensure efficient administration and taxation, often allowing local laws to coexist with Roman law."}

In [14]:
result["result"]

"The Roman Empire's government consisted of three major elements: the central government, the military, and the provincial government. The central government included elected magistrates and appointed officials who managed different aspects of state affairs, while the military maintained control and order. Provincial governance involved local elites and officials to ensure efficient administration and taxation, often allowing local laws to coexist with Roman law."

In [15]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

("The Roman Empire's government consisted of three major elements: the central "
 'government, the military, and the provincial government. The central '
 'government included elected magistrates and appointed officials who managed '
 'different aspects of state affairs, while the military maintained control '
 'and order. Provincial governance involved local elites and officials to '
 'ensure efficient administration and taxation, often allowing local laws to '
 'coexist with Roman law.')


In [16]:
question = "How was the education in roman empire?"
result = qa_chain({"query": question })
pp.pprint(result["result"])

('Education in the Roman Empire was moral and practical, aimed at instilling '
 'Roman values and often provided by parents or through apprenticeships. '
 'Formal education, available primarily to those who could afford it, included '
 'primary schooling in reading, writing, and arithmetic, with higher education '
 'focusing on literature, rhetoric, and philosophy. Schools became more '
 'numerous during the Empire, and education was considered essential for '
 'career advancement and social mobility, particularly for the urban elites.')
